# 04 - Topic Analysis (Refusal vs Acceptance)
Compare refusal vs acceptance corpora with bag-of-words/LDA and reuse migrated topic assets to avoid recomputing.

**Goals**
- Load Perspective-scored imitation bundle and attach refusal markers.
- Split per-conversation documents into refusal vs acceptance buckets.
- Reuse migrated LDA runs (Count/TF-IDF) and include a small rerun hook.

In [ ]:
from pathlib import Path
import pickle
from typing import Dict, List, Optional, Sequence, Union

import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from utils.data_io import load_df_list_pickle, flatten_conversation_bundles, describe_bundle

In [ ]:
# Paths and toggles
PROJECT_ROOT = Path.cwd()
ASSETS_PROCESSED = PROJECT_ROOT / "assets" / "processed"
ASSETS_TOPICS = ASSETS_PROCESSED / "topics"

PERSPECTIVE_PATH = ASSETS_PROCESSED / "combat_threads_with_perspective.pkl"

LDA_TFIDF_REF = ASSETS_TOPICS / "lda_results_tfidf_ref.pkl"
LDA_TFIDF_ACC = ASSETS_TOPICS / "lda_results_tfidf_acc.pkl"
LDA_COUNT_REF = ASSETS_TOPICS / "lda_results_count_ref.pkl"
LDA_COUNT_ACC = ASSETS_TOPICS / "lda_results_count_acc.pkl"
LDA_MISC = [
    ASSETS_TOPICS / "lda_results_tfidf.pkl",
    ASSETS_TOPICS / "lda_results_counter.pkl",
    ASSETS_TOPICS / "lda_results_updated.pkl",
]

REFUSAL_THRESHOLD = 0.1  # mark a conversation as refusal-heavy when > 10% turns refused
TEXT_COLUMN = "text"    # default to original user utterances; switch to "imm_1" to analyze imitation output
RUN_LDA = False          # set True to recompute a small LDA sweep locally
N_TOP_WORDS = 12
STOP_WORDS = ["gt", "people"]

ASSETS_TOPICS.mkdir(parents=True, exist_ok=True)
ASSETS_PROCESSED, ASSETS_TOPICS

### Asset manifest
List the perspective bundle plus migrated topic assets (copied from `../Raja/revised_convo/`).

In [ ]:
manifest = [
    {
        "role": "input",
        "path": PERSPECTIVE_PATH,
        "note": "Imitation + Perspective bundle (list of DataFrames).",
    },
    {
        "role": "resume_optional",
        "path": LDA_TFIDF_REF,
        "note": "TF-IDF LDA on refusal-heavy conversations (source: Raja/revised_convo/lda_results_tfidf_ref.pkl).",
    },
    {
        "role": "resume_optional",
        "path": LDA_TFIDF_ACC,
        "note": "TF-IDF LDA on accepted conversations (source: Raja/revised_convo/lda_results_tfidf_acc.pkl).",
    },
    {
        "role": "resume_optional",
        "path": LDA_COUNT_REF,
        "note": "Count-vector LDA on refusal-heavy conversations (source: Raja/revised_convo/lda_results_count_ref.pkl).",
    },
    {
        "role": "resume_optional",
        "path": LDA_COUNT_ACC,
        "note": "Count-vector LDA on accepted conversations (source: Raja/revised_convo/lda_results_count_acc.pkl).",
    },
]
for extra in LDA_MISC:
    manifest.append(
        {
            "role": "archive_optional",
            "path": extra,
            "note": f"Additional LDA sweep ({extra.name}) migrated from Raja/revised_convo.",
        }
    )
manifest_df = pd.DataFrame(manifest)
manifest_df["exists"] = manifest_df["path"].apply(Path.exists)
manifest_df

### Load and label the perspective bundle
Flatten into a DataFrame, attach a refusal flag (`is_refusal = not imm_1_check`), and keep a text column ready for topic modeling.

In [ ]:
imitation_bundle = load_df_list_pickle(PERSPECTIVE_PATH)
print("bundle summary:", describe_bundle(imitation_bundle))

flat = flatten_conversation_bundles(imitation_bundle)
flat["is_refusal"] = ~flat["imm_1_check"].astype(bool)
flat["text_for_topics"] = flat[TEXT_COLUMN].fillna("").astype(str)

print("rows:", len(flat))
print(flat["is_refusal"].value_counts(normalize=True))
flat.sample(5, random_state=0)[["conversation_idx", "text", "imm_1", "imm_1_check", "is_refusal"]]


### Conversation-level documents
Collapse each conversation into a single document for topic modeling and mark refusal-heavy conversations using `REFUSAL_THRESHOLD`.

In [ ]:
def build_conversation_docs(df: pd.DataFrame, text_col: str, refusal_threshold: float) -> pd.DataFrame:
    grouped = (
        df.groupby("conversation_idx")
        .agg(
            doc=(text_col, lambda s: " ".join(s.dropna().astype(str))),
            refusal_rate=("is_refusal", "mean"),
            row_count=("is_refusal", "size"),
        )
        .reset_index()
    )
    grouped["is_refusal_conversation"] = grouped["refusal_rate"] > refusal_threshold
    return grouped

conversation_docs = build_conversation_docs(flat, text_col="text_for_topics", refusal_threshold=REFUSAL_THRESHOLD)
print(conversation_docs["is_refusal_conversation"].value_counts())
conversation_docs.head()

### Corpus splits (refusal vs acceptance)
These lists feed into the precomputed LDA runs; swap `TEXT_COLUMN` to switch between model outputs and original text.

In [ ]:
docs_refusal = conversation_docs.loc[conversation_docs["is_refusal_conversation"], "doc"].tolist()
docs_accept = conversation_docs.loc[~conversation_docs["is_refusal_conversation"], "doc"].tolist()

print(f"refusal docs: {len(docs_refusal)} | accept docs: {len(docs_accept)}")
{
    "refusal_example": docs_refusal[0][:200] if docs_refusal else "",
    "accept_example": docs_accept[0][:200] if docs_accept else "",
}


### Optional: run LDA locally (heavy)
Keeps API-compatible outputs with the migrated pickles. Flip `RUN_LDA = True` and adjust `topic_range` to re-fit with Count/TF-IDF.

In [ ]:
def extract_top_words(model: LatentDirichletAllocation, vectorizer: Union[CountVectorizer, TfidfVectorizer], top_n: int) -> List[List[str]]:
    feature_names = vectorizer.get_feature_names_out()
    topics: List[List[str]] = []
    for topic in model.components_:
        top_ids = topic.argsort()[:-top_n - 1:-1]
        topics.append([feature_names[i] for i in top_ids])
    return topics


def run_lda_sweep(
    docs: Sequence[str],
    vectorizer: Union[CountVectorizer, TfidfVectorizer],
    topic_counts: Sequence[int],
    top_words: int,
    random_state: int = 0,
) -> List[Dict]:
    results: List[Dict] = []
    matrix = vectorizer.fit_transform(docs)
    for n_topics in topic_counts:
        lda = LatentDirichletAllocation(
            n_components=n_topics,
            learning_method="batch",
            max_iter=30,
            random_state=random_state,
        )
        lda.fit(matrix)
        results.append(
            {
                "n_topics": n_topics,
                "model": lda,
                "topics": extract_top_words(lda, vectorizer, top_n=top_words),
                "perplexity": lda.perplexity(matrix),
                "coherence": None,  # placeholder to stay compatible with migrated pickles
                "vectorizer": vectorizer,
            }
        )
    return results


topic_range = [5, 8, 10]
if RUN_LDA:
    tfidf_results_ref = run_lda_sweep(
        docs_refusal,
        TfidfVectorizer(stop_words=STOP_WORDS),
        topic_counts=topic_range,
        top_words=N_TOP_WORDS,
    )
    with (ASSETS_TOPICS / "lda_results_tfidf_ref_repro.pkl").open("wb") as fp:
        pickle.dump(tfidf_results_ref, fp)


### Load migrated LDA results and summarize
Quick glance at topic counts, vectorizers, and sample terms from each stored run.

In [ ]:
def load_lda_results(path: Path) -> Optional[List[Dict]]:
    if not path.exists():
        return None
    with path.open("rb") as fp:
        return pickle.load(fp)


loaded_results: Dict[str, Optional[List[Dict]]] = {
    "tfidf_ref": load_lda_results(LDA_TFIDF_REF),
    "tfidf_acc": load_lda_results(LDA_TFIDF_ACC),
    "count_ref": load_lda_results(LDA_COUNT_REF),
    "count_acc": load_lda_results(LDA_COUNT_ACC),
}
for extra in LDA_MISC:
    loaded_results[extra.stem] = load_lda_results(extra)

summary_rows = []
for name, runs in loaded_results.items():
    if not runs:
        continue
    for entry in runs:
        summary_rows.append(
            {
                "asset": name,
                "n_topics": entry.get("n_topics"),
                "vectorizer": entry.get("vectorizer").__class__.__name__ if entry.get("vectorizer") else None,
                "perplexity": entry.get("perplexity"),
                "coherence": entry.get("coherence"),
                "sample_terms": ", ".join(entry.get("topics", [[]])[0][:5]) if entry.get("topics") else "",
            }
        )
summary_df = pd.DataFrame(summary_rows)
summary_df.sort_values(["asset", "n_topics"]).head(12)


### Preview topics from the best runs
Pick the highest-coherence run per split and show its top terms.

In [ ]:
def choose_best(runs: Optional[List[Dict]]) -> Optional[Dict]:
    if not runs:
        return None
    return max(
        runs,
        key=lambda r: (
            r.get("coherence") if r.get("coherence") is not None else -1,
            -(r.get("perplexity") or 0),
        ),
    )


def topics_frame(entry: Dict) -> pd.DataFrame:
    return pd.DataFrame(
        {
            "topic": range(len(entry.get("topics", []))),
            "top_terms": [" ".join(words) for words in entry.get("topics", [])],
            "n_topics": entry.get("n_topics"),
            "vectorizer": entry.get("vectorizer").__class__.__name__ if entry.get("vectorizer") else None,
        }
    )

best_ref = choose_best(loaded_results.get("tfidf_ref"))
best_acc = choose_best(loaded_results.get("tfidf_acc"))

frames = []
if best_ref:
    frames.append(topics_frame(best_ref).assign(split="refusal"))
if best_acc:
    frames.append(topics_frame(best_acc).assign(split="accept"))

pd.concat(frames, ignore_index=True) if frames else "No LDA assets loaded."
